## 1. Importi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import csv
import os.path

## 2. Branje podatkov

In [ ]:
with open('../data/kd2019.csv', 'rt') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    head1 = next(reader)
    none_handler = lambda i : i or None
    main_data = np.array([none_handler(i) for i in [row for row in reader]])

### 2.1 Priprava podatkov

In [ ]:
cities, counts = np.unique(main_data[:, 19], return_counts=True)
dict_id_city = dict((id, str.lower(city)) for id, city in enumerate(cities))
dict_city_id = dict((str.lower(city), id) for id, city in enumerate(cities))

## 3. Štetje zločinov v posamezni občini

In [ ]:
# count number of occurences of each value in column
dict_cityId_crimeCount = dict((dict_city_id[str.lower(cities[i])], counts[i]) for i in range(len(cities)))

y_axis = np.array([str.capitalize(dict_id_city[cityId]) for cityId in dict_cityId_crimeCount.keys()])
x_axis = np.array([dict_cityId_crimeCount[cityId] for cityId in dict_cityId_crimeCount.keys()])

counts_argsort = np.argsort(x_axis)[::-1]

# make graph of number of occurences of each value in column
plt.figure(figsize=(20, 10))
plt.xticks(rotation=90)
plt.bar(y_axis[counts_argsort], x_axis[counts_argsort])
plt.show()

Že takoj na prvi pogled je razvidno, da se največ zločina zgodi v Ljubljani. Sigurno pa na podlagi teh rezultatov ne moremo odgovoriti na vprašanje, "Katera občina je najbolj varna/nevarna?". Dodal bom podatke za število prebivalcev v vsaki občini.

In [ ]:
with open('../data/stevilo_prebivalcev_po_obcinah_2019julij.csv', 'rt', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    head2 = next(reader)
    none_handler = lambda i : i or None
    count_per_city = np.array([none_handler(i) for i in [row for row in reader]])

In [ ]:
# if name contains / split by / and keep first part
count_per_city = [(str.split(row[0], '/')[0], row[1]) for row in count_per_city]
dict_cityId_popCount = dict([(dict_city_id[str.lower(count_per_city[i][0])], count_per_city[i][1]) for i in range(len(count_per_city)) if str.lower(count_per_city[i][0]) in dict_city_id.keys()])

# delete unknown values from cities we are watching
to_be_deleted = []
for id in dict_id_city.keys():
    if id not in dict_cityId_popCount.keys():
        to_be_deleted.append(id)

print(f"Izbrisani so bili podatki {[dict_id_city[id] for id in to_be_deleted]}, saj za njih ne poznamo prave občine, ali pa le ne poznamo števila prebivalcev.")

for id in to_be_deleted:
    del dict_city_id[dict_id_city[id]]
    del dict_id_city[id]

In [ ]:
# normalized data of crimes per city by population of city
crime_per_city_norm0 = np.array([tuple([cityId, float(dict_cityId_crimeCount[cityId]) / float(dict_cityId_popCount[cityId])]) for cityId in dict_id_city.keys()])

cityId, x_axis = zip(*crime_per_city_norm0)

x_axis_argsort = np.argsort(x_axis)[::-1]
x_axis = np.array(x_axis)
y_axis = np.array([str.capitalize(dict_id_city[id]) for id in cityId])


plt.figure(figsize=(20, 10))
plt.xticks(rotation=90)
plt.bar(y_axis[x_axis_argsort], x_axis[x_axis_argsort])
plt.show()

Murska sobota vrne največjo vrednost normaliziranega števila kriminala s številom prebivalcev. Zakaj?
Vse primere zločina v Murski Soboti bom zapisal v datoteko.

In [ ]:
ms_crime = np.array([i for i in main_data if str.lower(i[19]) == 'murska sobota'])

if (not os.path.exists('../data/kd2019_murska_sobota.csv')):
    open('../data/kd2019_murska_sobota.csv', 'x')

with open('../data/kd2019_murska_sobota.csv', 'w', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(head1)
    writer.writerows(ms_crime)